Student name:

Student ID:

Please use this notebook to implement task 2. Do not forget to replace IDnumber in the filename with your ID number.

Use the below information to create a Naive Bayes SPAM filter. Test your filter using the messages in new_emails

In [ ]:
#First, I ran tests using the provided data    : achieved 2/4 success

# I then increased the data-base and re-ran the tests: achieved 4/4 success

# I then ran additional tests with the larger data-base:  achieved 27/32 success

# ( unfortunately, I've lost the link where I got the additional ham/spam strings)

In [1]:
#   NOTATION
    #  P_ denotes 'probability of'          G_ denotes 'given'
    #
    # 'probability of a given b' --> P_a_G_b

In [27]:
previous_spam = ['send us your password', 'review our website', 'send your password', 'send us your account']
previous_ham = ['Your activity report','benefits physical activity', 'the importance vows']
new_emails = {'spam':['renew your password', 'renew your vows'], 'ham':['benefits of our account', 'the importance of physical activity']}

In [28]:
class SpamFilter():
    def __init__(self, previous_ham, previous_spam, new_emails):
        self.previous_ham = [phrase.lower() for phrase in previous_ham]
        self.previous_spam = [phrase.lower() for phrase in previous_spam]
        new_ham = [phrase.lower() for phrase in  new_emails['ham']]
        new_spam = [phrase.lower() for phrase in  new_emails['spam']]
        self.new_emails = {'ham': new_ham, 'spam':new_spam}
        self.all_previous_mail = self.previous_ham + self.previous_spam
        self.P_spam = len(previous_spam)/len(self.all_previous_mail)
        self.P_ham = len(previous_ham)/len(self.all_previous_mail)
        self.all_previous_words = self.getAllWords()
        self.P_previous_words_G_label = self.getP_words_G_label()           

    def getAllWords(self):
        allWords = []
        for phrase in self.all_previous_mail:
            for word in phrase.split():
                allWords.append(str(removePunctuation_fillerWords_and_caps(word)))
        allWords = list(dict.fromkeys(allWords))
        return allWords

    def getP_words_G_label(self):
        # eg. P(word|spam)
        wordProbabilities = {}
        minValue = 1/(len(self.previous_ham) + 2)
        for phrase in self.previous_ham:
            for word in self.all_previous_words:
                if not word in wordProbabilities:
                    wordProbabilities[word] = {'ham': minValue, 'spam': minValue}
                for i in phrase.split():
                    if word == i:
                        wordProbabilities[word]['ham'] += 1/(len(self.previous_ham) + 2)
        for phrase in self.previous_spam:
            for word in self.all_previous_words:
                # print(word)
                if not word in wordProbabilities:
                    wordProbabilities[word] = {'ham': minValue, 'spam': minValue}
                for i in phrase.split():
                    if word == i:
                        wordProbabilities[word]['spam'] += 1/len(self.previous_spam)
        return wordProbabilities

    def getP_phrase_G_label(self, phrase):
        phrase = phrase.lower()
        # eg. P(phrase|spam)
        probabilities = {'ham': 1, 'spam': 1}
        for word in self.P_previous_words_G_label:
            if word in phrase.split():
                probabilities['ham'] *= self.P_previous_words_G_label[word]['ham']
                probabilities['spam'] *= self.P_previous_words_G_label[word]['spam']
            else:
                probabilities['ham'] *= (1 - self.P_previous_words_G_label[word]['ham'])
                probabilities['spam'] *= (1 - self.P_previous_words_G_label[word]['spam'])
            # print(word)
            # print(probabilities)
        return probabilities

    def getP_label_G_phrase(self, phrase, printGuess = True):
        phrase = removePunctuation_fillerWords_and_caps(phrase)
        probabilities = {}
        numerator = self.getP_phrase_G_label(phrase)['spam']*self.P_spam
        denominator = (numerator + self.getP_phrase_G_label(phrase)['ham']*self.P_ham)
        if denominator == 0:
            probabilities['spam'] = 0.499
            probabilities['ham'] = 0.501
            if printGuess == True:
                self.printBayesDenominatorException(phrase)

        else:
            probabilities['spam'] = round(numerator/denominator, 2)
            probabilities['ham'] = round(1 - probabilities['spam'], 2)
        return probabilities
    
    def printBayesDenominatorException(self, phrase):
        # print("____________")
        # print("according to our data-set,")
        # print("P( \'"+str(phrase)+"\'|SPAM ) & P( \'"+str(phrase)+"\'|HAM ) = 0" )
        print("... Not enough data to determine result, so err with \'ham\'\n")
    
    def testMail(self):
        correctPrediction = 0
        index = 0
        for phrase in self.new_emails['ham']:
            probabilities = self.getP_label_G_phrase(phrase)
            print("ham   "+str(phrase)+"\n    " +str(probabilities))
            if probabilities['ham'] > probabilities['spam']:
                correctPrediction += 1
            index +=1
        for phrase in self.new_emails['spam']:
            probabilities = self.getP_label_G_phrase(phrase)
            print("spam  "+str(phrase)+"\n    " +str(probabilities))
            if probabilities['spam'] > probabilities['ham']:
                correctPrediction += 1
            index +=1
        print("\ncorrect predictions "+str(correctPrediction)+" / "+str(index))

    def testLargeMail(self):
        correctPrediction = 0
        index = 0
        guessCount = 0
        for phrase in self.new_emails['ham']:
            probabilities = self.getP_label_G_phrase(phrase, printGuess = False)
            if probabilities['ham'] > probabilities['spam']:
                correctPrediction += 1
            if probabilities['ham'] == 0.501:
                guessCount += 1
            index += 1
        for phrase in self.new_emails['spam']:
            probabilities = self.getP_label_G_phrase(phrase, printGuess = False)
            if probabilities['spam'] > probabilities['ham']:
                correctPrediction += 1
            if probabilities['ham'] == 0.501:
                guessCount += 1
            index += 1
        print("correct predictions "+str(correctPrediction)+" / "+str(index))

In [29]:
def removePunctuation_fillerWords_and_caps(my_str):
    # https://www.programiz.com/pythonprogramming/examples/remove-punctuation
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~,.-'''
    no_punct = ""
    for char in my_str:
        if char not in punctuations:
            no_punct = no_punct + char
    return removeFillerWords(no_punct.lower())

def removeFillerWords(my_str):
    fillerWords = [
        "as",
        "to",
        "and",
        "the",
        "a",
        "of",
        "our"
    ]
    usefulWords = [ word for word in my_str.split() if word not in fillerWords]
    my_str = ' '.join(usefulWords)
    return my_str

In [30]:
sf = SpamFilter(previous_ham, previous_spam, new_emails)
sf.testMail()

ham   benefits of our account
    {'spam': 0.0, 'ham': 1.0}
ham   the importance of physical activity
    {'spam': 0.0, 'ham': 1.0}
spam  renew your password
    {'spam': 0.42, 'ham': 0.58}
spam  renew your vows
    {'spam': 0.03, 'ham': 0.97}

correct predictions 2 / 4


In [31]:
print(sf.getAllWords())

['your', 'activity', 'report', 'benefits', 'physical', '', 'importance', 'vows', 'send', 'us', 'password', 'review', 'website', 'account']


In [32]:
print("for words & labels: P(word|label)")
print(sf.getP_words_G_label())

# with our small db, the word 'send' is very heavily weighted towards spam. This plays a large role in the low detection rate abovef or the strings "renew your password" & "renew your vows"

for words & labels: P(word|label)
{'your': {'ham': 0.4, 'spam': 0.95}, 'activity': {'ham': 0.6000000000000001, 'spam': 0.2}, 'report': {'ham': 0.4, 'spam': 0.2}, 'benefits': {'ham': 0.4, 'spam': 0.2}, 'physical': {'ham': 0.4, 'spam': 0.2}, '': {'ham': 0.2, 'spam': 0.2}, 'importance': {'ham': 0.4, 'spam': 0.2}, 'vows': {'ham': 0.4, 'spam': 0.2}, 'send': {'ham': 0.2, 'spam': 0.95}, 'us': {'ham': 0.2, 'spam': 0.7}, 'password': {'ham': 0.2, 'spam': 0.7}, 'review': {'ham': 0.2, 'spam': 0.45}, 'website': {'ham': 0.2, 'spam': 0.45}, 'account': {'ham': 0.2, 'spam': 0.45}}


In [33]:
print("P(phrase|label)")
print(sf.getP_phrase_G_label('renew your vows'))

print("\nP(label|phrase)")
print(sf.getP_label_G_phrase('renew your vows'))
#This one's wrong

P(phrase|label)
{'ham': 0.0017394617548800004, 'spam': 3.7290147840000066e-05}

P(label|phrase)
{'spam': 0.03, 'ham': 0.97}


In [34]:
# Below, the previous test cases and more sample data are added to the db, then the saame tests are re run, and then new tests are run

In [35]:
moreSpamSamples = new_emails['spam'] +  [
    "You are receiving this letter because you have expressed an interest in ",
    "receiving information about online business opportunities, If this is ",
    "erroneous then please accept my most sincere apology, This is a one-time ",
    "mailing, so no removal is necessary,",
    "If you've been burned, betrayed, and back-stabbed by multi-level marketing, ",
    "MLM, then please read this letter, It could be the most important one that ",
    "has ever landed in your Inbox,",
    "MULTI-LEVEL MARKETING IS A HUGE MISTAKE FOR MOST PEOPLE",
    "MLM has failed to deliver on its promises for the past 50 years, The pursuit ",
    "of the MLM Dream has cost hundreds of thousands of people their friends, ",
    "their fortunes and their sacred honor, The fact is that MLM is fatally ",
    "flawed, meaning that it CANNOT work for most people,",
    "The companies and the few who earn the big money in MLM are NOT going to ",
    "tell you the real story, FINALLY, there is someone who has the courage to ",
    "cut through the hype and lies and tell the TRUTH about MLM,",
    "HERE'S GOOD NEWS",
    "There IS an alternative to MLM that WORKS, and works BIG! If you haven't yet ",
    "abandoned your dreams, then you need to see this, Earning the kind of income ",
    "you've dreamed about is easier than you think!",
    "With your permission, I'd like to send you a brief letter that will tell you "
    ]

moreHamSamples = new_emails['ham'] + [
 "which is probably related to my not having an unseen sequence anywhere",
 "(certainly not in inbox) , I read all of my outstanding mail before I",
 "tried this new exmh",
 "Second, I've been used to having a key binding which was to Msg_MarkUnseen",
 "which doesn't seem to exist any more, and I'm not sure what I should replace",
 "that with, There's obviously a way as the Sequences menu does this,",
 "The Mark Unseen menu entry in the message More menu is still wanting",
 "that function as well,,,",
 "For those who have other sequences defined, the window will widen to",
 "display the other sequences,",
 "Any chance of having that lengthen instead? I like all my exmh stuff",
 "in nice columns (fits the display better), That is, I use the detached",
 "folder list, one column, The main exmh window takes up full screen,",
 "top to bottom, but less than half the width, etc,,,",
 "I have space for more sequences, in the unseen window, as long as they",
 "remain once nice narrow window (best would be if the sequences could be",
 "ordered by some preference, then ones which didn't fit would just fall",
 "off the bottom, and not be shown),",
 "I'd also prefer it if that window had no unusual background colouring,",
 "just one constant colour , I have been running the unseen window with",
 "background black, on a root window that is all black, with no borders or",
 "other decorations, but made sticky , the appearance is just like the"
    ]

# see if these help for etivity test-emails.
#The tests are being re-run with the addition of a single spam & ham line every time to see the effect
for H in range(len(moreHamSamples)):
    sf2 = SpamFilter(previous_ham + moreHamSamples[0:H], previous_spam + moreSpamSamples[0:H], new_emails)
    sf2.testMail()


# as we can see, with the additon of more data, the SpamFilter could recognize more (all)test cases.

ham   benefits of our account
    {'spam': 0.0, 'ham': 1.0}
ham   the importance of physical activity
    {'spam': 0.0, 'ham': 1.0}
spam  renew your password
    {'spam': 0.42, 'ham': 0.58}
spam  renew your vows
    {'spam': 0.03, 'ham': 0.97}

correct predictions 2 / 4
ham   benefits of our account
    {'spam': 0.0, 'ham': 1.0}
ham   the importance of physical activity
    {'spam': 0.0, 'ham': 1.0}
spam  renew your password
    {'spam': 0.95, 'ham': 0.05}
spam  renew your vows
    {'spam': 0.33, 'ham': 0.67}

correct predictions 3 / 4
ham   benefits of our account
    {'spam': 0.0, 'ham': 1.0}
ham   the importance of physical activity
    {'spam': 0.0, 'ham': 1.0}
spam  renew your password
    {'spam': 0.99, 'ham': 0.01}
spam  renew your vows
    {'spam': 0.89, 'ham': 0.11}

correct predictions 4 / 4
ham   benefits of our account
    {'spam': 0.02, 'ham': 0.98}
ham   the importance of physical activity
    {'spam': 0.0, 'ham': 1.0}
spam  renew your password
    {'spam': 0.98, 'ham': 0

In [36]:
# try more test data

# A likely of bias for these test cases are that they are taken from the same two ham and spam emails as the above sample-emails.
moreTests = {
    'ham': [
        "folders with unseen messages (and their counts) are written into the",
        "root window (because it is sticky, this small display follows me around",
        "and do I can see when new mail needs processing),",
        "I also find that I tend to have a bunch of sequences that only ever occur",
        "in one folder (some I had forgotten I ever created), So in addition to",
        "the sequences to always show and sequences to never show, a",
        "preference to only show sequences that occur in more than one folder",
        "would be useful, and then have the sequences that occor only in the",
        "folder I'm visiting appear in the list when that folder is current,",
        "This is just to keep the list size somewhat manageable while remaining",
        "productive (I quite often use a sequence to remember a particular message",
        "in a folder , the name is used only there, and only for one message,",
        "it gives me a handle on the message which remains as the folder is"
    ],
    'spam': new_emails['spam'] + [
        "WHY MLM doesn't work for most people and will then introduce you to ",
        "something so new and refreshing that you'll wonder why you haven't heard of ",
        "this before,",
        "I promise that there will be NO unwanted follow up, NO sales pitch, no one ",
        "will call you, and your email address will only be used to send you the ",
        "information, Period,",
        "To receive this free, life-changing information, simply click Reply, type ",
        "Send Info in the Subject box and hit Send, I'll get the information to you ",
        "within 24 hours, Just look for the words MLM WALL OF SHAME in your Inbox,",
        "Cordially",
        "P,S, Someone recently sent the letter to me and it has been the most ",
        "eye-opening, financially beneficial information I have ever received, I ",
        "honestly believe that you will feel the same way once you've read it, And ",
        "it's FREE!",
        "This email is NEVER sent unsolicited.  THIS IS NOT SPAM. You are receiving  ",
        "this email because you EXPLICITLY signed yourself up to our list with our",
        "online signup form or through use of our FFA Links Page and E-MailDOM "
    ]
}

for H in range(len(moreHamSamples)):
        print("\ndb stores "+str(H+4)+"x2 sample strings")
        sf2 = SpamFilter(previous_ham + moreHamSamples[0:H], previous_spam + moreSpamSamples[0:H], moreTests)
        sf2.testLargeMail()


db stores 4x2 sample strings
correct predictions 14 / 32

db stores 5x2 sample strings
correct predictions 15 / 32

db stores 6x2 sample strings
correct predictions 17 / 32

db stores 7x2 sample strings
correct predictions 20 / 32

db stores 8x2 sample strings
correct predictions 20 / 32

db stores 9x2 sample strings
correct predictions 19 / 32

db stores 10x2 sample strings
correct predictions 19 / 32

db stores 11x2 sample strings
correct predictions 22 / 32

db stores 12x2 sample strings
correct predictions 20 / 32

db stores 13x2 sample strings
correct predictions 24 / 32

db stores 14x2 sample strings
correct predictions 20 / 32

db stores 15x2 sample strings
correct predictions 21 / 32

db stores 16x2 sample strings
correct predictions 21 / 32

db stores 17x2 sample strings
correct predictions 20 / 32

db stores 18x2 sample strings
correct predictions 20 / 32

db stores 19x2 sample strings
correct predictions 20 / 32

db stores 20x2 sample strings
correct predictions 21 / 32

db

In [37]:
# From the additional experiment above, I gather that my spam filter algorithm can achieve quite a high success rate, given prior ( and sole ) exposure to a given writer.
# success rate peaks at 25 / 32, for which the db stored 27 ham & 27 spam sample strings.
# it is noteworthy that in this case a larger db doesn't necessarily mean more accurate results, as the success rate peaks at 14x2 strings of previous ham/spam.
# Given more time I would investigate if the addition of 'ham' / 'spam' email strings by a different author would confuse the algorithm.
# I would also be interested in storing entire emails in the db, then trying to recognize spam emails as a whole, rathen than just isolated phrases.